In [ ]:
import sys

sys.path.insert(0, '../')

In [3]:
from common.common import create_folder
from common.pytorch import load_model
import pytorch_pretrained_bert as Bert
from model.utils import age_vocab
from common.common import load_obj
from dataLoader.MLM import MLMLoader
from torch.utils.data import DataLoader
import pandas as pd
from model.MLM import BertForMaskedLM
from model.optimiser import adam
import sklearn.metrics as skm
import numpy as np
import torch
import time
import torch.nn as nn
import os

In [4]:
class BertConfig(Bert.modeling.BertConfig):
    def __init__(self, config):
        super(BertConfig, self).__init__(
            vocab_size_or_config_json_file=config.get('vocab_size'),
            hidden_size=config['hidden_size'],
            num_hidden_layers=config.get('num_hidden_layers'),
            num_attention_heads=config.get('num_attention_heads'),
            intermediate_size=config.get('intermediate_size'),
            hidden_act=config.get('hidden_act'),
            hidden_dropout_prob=config.get('hidden_dropout_prob'),
            attention_probs_dropout_prob=config.get('attention_probs_dropout_prob'),
            max_position_embeddings=config.get('max_position_embedding'),
            initializer_range=config.get('initializer_range'),
        )
        self.seg_vocab_size = config.get('seg_vocab_size')
        self.age_vocab_size = config.get('age_vocab_size')


class TrainConfig(object):
    def __init__(self, config):
        self.batch_size = config.get('batch_size')
        self.use_cuda = config.get('use_cuda')
        self.max_len_seq = config.get('max_len_seq')
        self.train_loader_workers = config.get('train_loader_workers')
        self.test_loader_workers = config.get('test_loader_workers')
        self.device = config.get('device')
        self.output_dir = config.get('output_dir')
        self.output_name = config.get('output_name')
        self.best_name = config.get('best_name')

In [5]:
file_config = {
    'vocab': r'my_data/vocab.pkl',  # vocabulary idx2token, token2idx
    'data': r'my_data.test.csv',  # formated data
    'model_path': r'my_data',  # where to save model
    'model_name': 'my-custom-mlm-model',  # model name
    'file_name': r'my_data/logs.txt',  # log path
}
create_folder(file_config['model_path'])

In [6]:
global_params = {
    'max_seq_len': 64,
    'max_age': 110,
    'month': 1,
    'age_symbol': None,
    'min_visit': 5,
    'gradient_accumulation_steps': 1
}

optim_param = {
    'lr': 3e-5,
    'warmup_proportion': 0.1,
    'weight_decay': 0.01
}

train_params = {
    'batch_size': 256,
    'use_cuda': True,
    'max_len_seq': global_params['max_seq_len'],
    'device': 'cuda:0'
}

In [7]:
BertVocab = load_obj(file_config['vocab'])
ageVocab, _ = age_vocab(max_age=global_params['max_age'], mon=global_params['month'],
                        symbol=global_params['age_symbol'])

In [8]:
# TODO: should return it?
#data = pd.read_parquet(file_config['data'])

data = pd.read_csv(file_config['data'], sep='\t')
#data.columns = ['id', 'code', 'age', 'year', 'static', 'explabel', 'label']
# remove patients with visits less than min visit

# TODO: should return it?
#data['length'] = data['caliber_id'].apply(lambda x: len([i for i in range(len(x)) if x[i] == 'SEP']))
#data = data[data['length'] >= global_params['min_visit']]
#data = data.reset_index(drop=True)

In [9]:
data.head()

,Unnamed: 0,code,age,year,static,explabel,label
0,88412,"['disease1', 'SEP', 'medication1', 'medication...","['600', '600', '650', '650', '650', '650', '65...","['1994', '1994', '1998', '1998', '1998', '1998...","[1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, ...",0,0
1,282358,"['disease1', 'SEP', 'medication1', 'medication...","['600', '600', '650', '650', '650', '650', '65...","['1994', '1994', '1998', '1998', '1998', '1998...","[1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, ...",0,0
2,76509,"['disease1', 'SEP', 'medication1', 'medication...","['600', '600', '650', '650', '650', '650', '65...","['1994', '1994', '1998', '1998', '1998', '1998...","[1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, ...",0,0
3,172790,"['disease1', 'SEP', 'medication1', 'medication...","['600', '600', '650', '650', '650', '650', '65...","['1994', '1994', '1998', '1998', '1998', '1998...","[1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, ...",0,0
4,450220,"['disease1', 'SEP', 'medication1', 'medication...","['600', '600', '650', '650', '650', '650', '65...","['1994', '1994', '1998', '1998', '1998', '1998...","[1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, ...",0,0


In [10]:
# TODO: should return it?
#Dset = MLMLoader(data, BertVocab['token2idx'], ageVocab, max_len=train_params['max_len_seq'], code='caliber_id')
Dset = MLMLoader(data, BertVocab['token2idx'], ageVocab, max_len=train_params['max_len_seq'])
trainload = DataLoader(dataset=Dset, batch_size=train_params['batch_size'], shuffle=True, num_workers=3)

In [11]:
model_config = {
    'vocab_size': len(BertVocab['token2idx'].keys()),  # number of disease + symbols for word embedding
    'hidden_size': 288,  # word embedding and seg embedding hidden size
    'seg_vocab_size': 2,  # number of vocab for seg embedding
    'age_vocab_size': len(ageVocab.keys()),  # number of vocab for age embedding
    'max_position_embedding': train_params['max_len_seq'],  # maximum number of tokens
    'hidden_dropout_prob': 0.1,  # dropout rate
    'num_hidden_layers': 6,  # number of multi-head attention layers required
    'num_attention_heads': 12,  # number of attention heads
    'attention_probs_dropout_prob': 0.1,  # multi-head attention dropout rate
    'intermediate_size': 512,  # the size of the "intermediate" layer in the transformer encoder
    'hidden_act': 'gelu',
    # The non-linear activation function in the encoder and the pooler "gelu", 'relu', 'swish' are supported
    'initializer_range': 0.02,  # parameter weight initializer range
}

In [12]:
conf = BertConfig(model_config)
model = BertForMaskedLM(conf)

In [13]:
#TODO: remove this.
train_params['device'] = 'cpu'

In [14]:
model = model.to(train_params['device'])
optim = adam(params=list(model.named_parameters()), config=optim_param)

t_total value of -1 results in schedule not being applied


In [15]:
def cal_acc(label, pred):
    logs = nn.LogSoftmax()
    label = label.cpu().numpy()
    ind = np.where(label != -1)[0]
    truepred = pred.detach().cpu().numpy()
    truepred = truepred[ind]
    truelabel = label[ind]
    truepred = logs(torch.tensor(truepred))
    outs = [np.argmax(pred_x) for pred_x in truepred.numpy()]
    precision = skm.precision_score(truelabel, outs, average='micro')
    return precision

In [19]:
def train(e, loader):
    tr_loss = 0
    temp_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    cnt = 0
    start = time.time()

    for step, batch in enumerate(loader):
        cnt += 1
        batch = tuple(t.to(train_params['device']) for t in batch)
        age_ids, input_ids, posi_ids, segment_ids, attMask, masked_label = batch
        loss, pred, label = model(input_ids, age_ids, segment_ids, posi_ids, attention_mask=attMask,
                                  masked_lm_labels=masked_label)
        if global_params['gradient_accumulation_steps'] > 1:
            loss = loss / global_params['gradient_accumulation_steps']
        loss.backward()

        temp_loss += loss.item()
        tr_loss += loss.item()

        nb_tr_examples += input_ids.size(0)
        nb_tr_steps += 1

        if step % 200 == 0:
            print(
                "epoch: {}\t| cnt: {}\t|Loss: {}\t| precision: {:.4f}\t| time: {:.2f}".format(e, cnt, temp_loss / 2000,
                                                                                              cal_acc(label, pred),
                                                                                              time.time() - start))
            temp_loss = 0
            start = time.time()

        if (step + 1) % global_params['gradient_accumulation_steps'] == 0:
            optim.step()
            optim.zero_grad()

    print("** ** * Saving fine - tuned model ** ** * ")
    model_to_save = model.module if hasattr(model, 'module') else model  # Only save the model it-self
    create_folder(file_config['model_path'])
    output_model_file = os.path.join(file_config['model_path'], file_config['model_name'])

    torch.save(model_to_save.state_dict(), output_model_file)

    cost = time.time() - start
    return tr_loss, cost

In [20]:
f = open(os.path.join(file_config['model_path'], file_config['file_name']), "w")
f.write('{}\t{}\t{}\n'.format('epoch', 'loss', 'time'))
data_len = data.shape[0]

for e in range(50):
    loss, time_cost = train(e, trainload)
    loss = loss / data_len
    f.write('{}\t{}\t{}\n'.format(e, loss, time_cost))
f.close()    

c:\users\ofir4\pycharmprojects\behrt\venv\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  


epoch: 0	| cnt: 1	|Loss: 4.131817445158959e-05	| precision: 0.9367	| time: 19.88
** ** * Saving fine - tuned model ** ** * 


c:\users\ofir4\pycharmprojects\behrt\venv\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  


epoch: 1	| cnt: 1	|Loss: 1.6251960769295692e-05	| precision: 0.9872	| time: 19.24
** ** * Saving fine - tuned model ** ** * 


c:\users\ofir4\pycharmprojects\behrt\venv\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  


...

c:\users\ofir4\pycharmprojects\behrt\venv\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  


epoch: 4	| cnt: 1	|Loss: 5.0653215497732165e-05	| precision: 0.9189	| time: 18.52
** ** * Saving fine - tuned model ** ** * 


c:\users\ofir4\pycharmprojects\behrt\venv\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  


epoch: 5	| cnt: 1	|Loss: 2.26371344178915e-05	| precision: 0.9595	| time: 18.50


KeyboardInterrupt: 